In [1]:
import sys
sys.path.append('./../gp_sinkhorn/')
sys.path.append('./..')
from SDE_solver import solve_sde_RK
from gp_sinkhorn.utils import plot_trajectories_2
from MLE_drift import *

import copy
import torch
import math
import numpy as np

from celluloid import Camera
from IPython.display import HTML
import matplotlib.pyplot as plt
import pickle as pkl

In [2]:
mocap = pkl.load(open("./../assets/motion_data.pkl", "rb"))
#mocap = pkl.load(open("./motion_data (4).pkl", "rb"))


In [3]:
full_dict, downsampled_dict, key_ = mocap


In [4]:
key_dict = {}
for i in key_:
    if i[0] != '29':
        key_dict[i[0]+i[1]] = [i[2],i[3]]
    else:
        print(i)

['29', '07', 515, 533]
['29', '02', 486, 505]


#  Estimating speed boundaries from position only 


The next cell shows you how to obtain the speed timeseries and subsequently speed boundaries given only position time series as inputs which is needed for the MOCAP data. Once obtain position and speed boundaries must be concatenated respectively before feeding to IPFP. Virtually this is all the setup we need + the motion priors illustrated in the earlier part of this notebook.


In [5]:
from scipy.signal import savgol_filter


def estimate_boundary_velocities(X, window=7, poly=5, mode='nearest'):
    n, t, d = X.shape
    
    V = np.zeros((n,t,d))
    
    Xnp = np.array(X)
    
    for i in range(n):
        for dj in range(d):
            V[i, :, dj] = savgol_filter(Xnp[i, :, dj], window, poly, mode=mode, deriv=1, delta=0.01)
    
    return torch.tensor(V).double()


# Vts = estimate_boundary_velocities(Xts[:,:,:dim*particles], window=3, poly=2, mode="interp")

In [6]:
out_mat_pos = []
out_mat_vel = []

P_0 = []
P_1 = []


sensor_no = 3

for k, v in full_dict.items():
    for k1, v1 in v.items():
        try:
            out_mat_vel.append(
                estimate_boundary_velocities(torch.tensor([v1]), window=5, poly=3, mode="interp")[0]
            )

            out_mat_pos.append(
                torch.tensor([v1])[0]
            )
            start,end = key_dict[k+k1][0],key_dict[k+k1][1]
            
            
            P_0.append(torch.cat((out_mat_pos[-1][start][:sensor_no], out_mat_vel[-1][start][:sensor_no])))

            P_1.append(torch.cat((out_mat_pos[-1][end][:sensor_no], out_mat_vel[-1][end][:sensor_no])))
        except:
            print("Key ",k,k1," has been cut")


# out_mat = torch.tensor(out_mat).double?()
# print(P_0)
P_0 = torch.vstack(P_0)
P_1 = torch.vstack(P_1)

P = torch.vstack([P_0,P_1])
P_0 = (P_0-P.mean(axis=0))/P.std(axis=0)
P_1 = (P_1-P.mean(axis=0))/P.std(axis=0)

Key  29 02  has been cut
Key  29 07  has been cut
Key  30 07  has been cut


In [7]:
def solve_sde_RK_s(b_drift=None, sigma=None, X0=None, dt=1.0, N=100, t0=0.0,
                 theta=None, noise=False, forwards=True):
    """
            Euler Mayurama method
    Syntax:
    ----------
    solve_sde(b_drift=None, sigma=None, X0=None, dt=None, N=100, t0=0, DW=None)
    Parameters:
    ----------
        b_drift  : Time dependan drift, the X state (with last dimension as time)
                defines the differential equation.
        sigma  : a  constant volatility
        X0    : Initial conditions of the SDE. Mandatory for SDEs
                with variables > 1 (default: gaussian np.random)
        dt    : The timestep of the solution
                (default: 1)
        N     : The number of timesteps (defines the length of the timeseries)
                (default: 100)
        t0    : The initial time of the solution
                (default: 0)

    """
    N = int(N) + 1
    if b_drift is None or sigma is None:
        raise ValueError("Error: SDE not defined.")

    n, d, *_ = X0.shape

    T = torch.tensor(dt * N)
    DWs = torch.empty((n, N - 1, d)).normal_(mean=0, std=1) * math.sqrt(dt)

    Y, ti = torch.zeros((n, N, d + 1)).double(), torch.arange(N).double() * dt + t0
    t0rep = (
        t0 * torch.ones((X0.shape[0], 1)).double() if forwards
        else (T - t0) * torch.ones((X0.shape[0], 1)).double()
    )
    Y = torch.cat((X0, t0rep), axis=1)[:, None, :]
    T = dt * N
    for n in range(N - 1):
        t = ti[n + 1]  # 1)
        b, DW_n = b_drift(Y[:, n, :]), DWs[:, n, :]
        
#         import pdb; pdb.set_trace()
        newY = (
                Y[:, n, :-1] + b * dt + sigma * DW_n
        )

        trep = (
            t.repeat(newY.shape[0]).reshape(-1, 1) if forwards
            else T - t.repeat(newY.shape[0]).reshape(-1, 1)
        )
        # print(trep)
        tocat = torch.cat((newY, trep), dim=1)[:, None, :]
        Y = torch.cat((Y, tocat), dim=1)
    if torch.isnan(Y).any() or torch.isinf(Y).any(): import pdb; pdb.set_trace()

    return ti, Y


# Generating Noisy Spring Based data

Here we simmulate some OU data using the EM SDE Solver . We will use this data to fit forwards and backwards drifts.

The system we  are implementing here is :

\begin{align}
d\begin{pmatrix}
\mathbf{x} \\
\mathbf{v}
\end{pmatrix} = \begin{pmatrix}
\mathbf{v} \\
\mathrm{\mathbf{K}} \mathbf{x} - \beta\mathbf{v} 
\end{pmatrix}dt + \gamma\begin{pmatrix}
0\\
1
\end{pmatrix}dW_t
\end{align}

Where we try the following settings for  $\mathrm{\mathbf{K}}$ :

\begin{align}
\mathrm{\mathbf{K}} = \begin{pmatrix}
-1 & -0.5 \\
-0.5 & -1
\end{pmatrix}, \quad \mathrm{\mathbf{K}} = \begin{pmatrix}
-1 & 0 \\
0 & -1
\end{pmatrix}, \quad \mathrm{\mathbf{K}} = \begin{pmatrix}
0 & 0 \\
0 & 0
\end{pmatrix}
\end{align}

This is only a 2x2 example however for block structure (many particles) we can imppose a block diag structure for example (correlates within a particles dimensions but not across particles).  The last setting where the spring matrix is 0 corresponds to a free particle experiencing no forces all we get out of this are some sound regularity constraints for motion that vanilla Brownian dynamics fails to impose whilst Langevin dynamics succesfully imposes.

In [8]:
def create_prior_langevin_diffusion_coeficient(dim_times_particles=1, gamma=1):
    """
    Creates the diffussion across all dimensions and across al particles
    for a classical motion prior
    """
    zs = torch.zeros((dim_times_particles,1))
    ons = torch.ones((dim_times_particles,1))
    
    return gamma * torch.cat((zs, ons), axis=0).T


def create_dimension_correlation_matrix(dim=1, decoupled=True):
    """
    Creates the coupling matrix within the dimensions of a single particle
    """
    corr = torch.tensor(torch.eye(dim))
    if not decoupled:
        rows, col = torch.range(0,dim-1).long(), torch.range(0,dim-1).long()
        halfs = 0.5 * torch.ones((dim, dim))
        halfs[rows, col] = 0.0
        return corr + halfs
    return corr


def create_full_correlation_matrix(
    dim_no=1, particle_no=1, decoupled_dim=True, decoupled_particles=True
):
    """
    Ultimately this just creates a large matrix with 1s in the diagonal
    and 0.5's everywhere else however it can be more flexible and
    allow for block structure
    
    dim_no : number of dimensions of each particle / sensors
    particle_no: number of particles / sensors
    """
    
    # create the diagonal blocks
    dim_mat = create_dimension_correlation_matrix(dim_no, decoupled_dim)
    
    full_dim = dim_no * particle_no
    big_corr = (torch.zeros((full_dim, full_dim)) if 
                decoupled_particles else 0.5 * torch.ones((full_dim, full_dim)) )
    
    # set block diagonals    
    for i in range(particle_no):
        big_corr[i * dim: (i+1) * dim, i * dim: (i+1) * dim] = torch.tensor(dim_mat)
    
    return big_corr


def create_prior_langevin_drift(
    dim_no=1, particle_no=1,
    free_particle=True, beta=1.0,
    decoupled_dim=True, decoupled_particles=True,
    hooks_constant = 0.01
):
    """
    This creates the linear drift coeficient for motion
    Set beta to 0 for undappened motion
    """
    full_dim = dim_no * particle_no
    # correlation / spring constant matrix with unit mass 
    K = (create_full_correlation_matrix(dim_no, particle_no, decoupled_dim, decoupled_particles)
        if not free_particle else torch.zeros((dim_no, particle_no))).double()
    
    K *= hooks_constant
    def linear_drift(phase_space_coords):
        x = phase_space_coords[:, :full_dim]
        v = phase_space_coords[:, full_dim:-1]

        xn = v
        vn = x.mm(K.T) - beta * v

        return torch.cat((xn, vn), axis=1)
    
    return linear_drift
    
    
    

In [9]:
num_samples = 15

# mu, std = torch.tensor([[np.pi/4.0] * particles, [np.pi/4.0]*particles ]), (0.1 * torch.eye(dim*particles) )
# X_0 = torch.distributions.MultivariateNormal(mu.double().flatten(), std.double()).sample((num_samples,)) #.reshape(num_samples, dim,-1)
# V_0 = torch.distributions.MultivariateNormal(mu.double().flatten(), std.double()).sample((num_samples,))

# P_0 = torch.cat((X_0, V_0), axis=1)
# print(P_0.shape)
#P_0 = (P_0 - P_0.mean(axis=0)) / P_0.std(axis=0)
#P_1 = (P_1 - P_1.mean(axis=0)) / P_1.std(axis=0)
P_0_time_aug = torch.cat((P_0, torch.zeros(P_0.shape[0],1)), axis=1)
# print(P_0_time_aug.shape)


In [10]:
dim = 3
particles = 1
gamma = 3
sigma_t = create_prior_langevin_diffusion_coeficient(dim * particles, gamma=gamma) 
beta  = 3 # Damping
free_particle = False
# this constant directly affects the sinusoids period
k = -14

# dt = 0.05
N = 100
dt = 1.0 / N
b_drift = create_prior_langevin_drift(
    dim_no=dim,
    particle_no=particles,
    beta=beta,
    free_particle=free_particle,
    hooks_constant=k,
    decoupled_dim=False, decoupled_particles=True
)

# Shape debug
b_drift(P_0_time_aug).shape, sigma_t.shape

 <ipython-input-8-db09ef1f8ddd>:16: UserWarning:To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 <ipython-input-8-db09ef1f8ddd>:18: UserWarning:torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
 <ipython-input-8-db09ef1f8ddd>:46: UserWarning:To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(torch.Size([12, 6]), torch.Size([1, 6]))

In [ ]:
sigma_tt = torch.tensor(sigma_t)
sigma_tt[sigma_tt==0] = 0.3

print(sigma_tt)
prior_X_0=torch.cat((P_1,P_0))


num_samples_prior = 100
mu, std = P_0.mean(axis=0), torch.diag(P_0.std(axis=0))
print(std.shape)
prior_X_0_1 = torch.distributions.MultivariateNormal(mu.double().flatten(), std.double()).sample((num_samples_prior,))

mu2, std2 = P_1.mean(axis=0), torch.diag(P_1.std(axis=0))
print(std2.shape)
prior_X_0_2 = torch.distributions.MultivariateNormal(mu2.double().flatten(), std2.double()).sample((num_samples_prior,))


prior_X_0f = torch.cat((prior_X_0_1,prior_X_0_2))




result = MLE_IPFP(
    P_0,P_1,
     N=N,prior_X_0=prior_X_0f,
    prior_drift=b_drift, iteration=10, sigma=(sigma_tt.flatten()),gp_mean_prior_flag=True, plot=True, langevin=True
)


tensor([[0.3000, 0.3000, 0.3000, 3.0000, 3.0000, 3.0000]])
torch.Size([6, 6])
torch.Size([6, 6])


 <ipython-input-11-cbad774fd8d1>:1: UserWarning:To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 c:\users\vargf\onedrive\documents\projects\gp_sinkhorn\gp_sinkhorn\GP.py:111: UserWarning:To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
